In [1]:
%load_ext autoreload
%autoreload 2

import helper_functions as hf
from imports import *
import importlib

num_available_cpus = multiprocessing.cpu_count()
print("Number of available CPUs:", num_available_cpus)

torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)
torch.set_default_tensor_type('torch.cuda.FloatTensor') if torch.cuda.is_available() else print ('cpu')

torch.set_num_threads(num_available_cpus)

print("Number of threads:", torch.get_num_threads())
print("Number of interop threads:", torch.get_num_interop_threads())

/nobackup/users/sambt/anaconda3/envs/quak/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of available CPUs: 80
Device = cuda:0
Number of threads: 80
Number of interop threads: 80


In [11]:
import re
plt.rcParams['font.size'] = 14
for d in os.listdir("test_data_forStats"):
    if "sigTrain" not in d:
        continue
    plt.figure(figsize=(10,10))
    training = re.search("sigTrain(.*)_bkgTrain",d).group(1)
    plt.title("Training: "+training)
    bkg_files = [f for f in os.listdir("test_data_forStats/"+d) if 'QCDBKG' in f]
    bkg_siglosses = []
    bkg_bkglosses = []
    for bf in bkg_files:
        arr = np.load("test_data_forStats/"+d+"/"+bf)
        bkg_siglosses.append(arr[1])
        bkg_bkglosses.append(arr[2])
        del arr
    bkg_siglosses = np.concatenate(bkg_siglosses,axis=0)
    bkg_bkglosses = np.concatenate(bkg_bkglosses,axis=0)
    plt.scatter(bkg_bkglosses[:100000],bkg_siglosses[:100000],s=2,label="QCDBKG")
    
    sig_files = [f for f in os.listdir("test_data_forStats/"+d) if 'QCDBKG' not in f]
    for sf in sig_files:
        name = "".join(sf.split("_")[1:]).split(".")[0]
        arr = np.load("test_data_forStats/"+d+"/"+sf)
        plt.scatter(arr[2][:10000],arr[1][:10000],s=2,label=name)
        del arr
    plt.xlim([0,50])
    plt.ylim([0,50])
    plt.legend(loc='upper right')
    plt.savefig("plots/QUAK_spaces/"+d+"/combined.png")
    plt.close()